In [106]:
import numpy as np
import pandas as pd
import pickle

In [6]:
def fetchIngredients():
    recipeIngredients = pd.read_csv('./CulinaryDB/04_Recipe-Ingredients_Aliases.csv')
    recipeIngredients["Entity ID"] = recipeIngredients["Entity ID"].astype(str)
#     names = recipeIngredients.groupby("Recipe ID").apply(lambda order: order['Original Ingredient Name'].tolist())
    aliased_names = recipeIngredients.groupby("Recipe ID").apply(lambda order: order['Aliased Ingredient Name'].tolist())
#     entity_ids = recipeIngredients.groupby("Recipe ID").apply(lambda order: order['Entity ID'].tolist())
    simpleIngredients = pd.read_csv('./CulinaryDB/02_Ingredients.csv')
    compoundIngredients = pd.read_csv('./CulinaryDB/03_Compound_Ingredients.csv')
    vocabulary = list()
    vocabulary.extend(list(simpleIngredients['Aliased Ingredient Name']))
    x = list(simpleIngredients['Ingredient Synonyms'])
    temp = []
    for i in x:
        z = i.split(";")
        z = [sub.strip() for sub in z]
        temp.extend(z)
    vocabulary.extend(temp)
    vocabulary.extend(list(compoundIngredients['Compound Ingredient Name']))
    y = list(compoundIngredients['Compound Ingredient Synonyms'])
    temp2 = []
    for i in y:
        z = i.split(";")
        z = [sub.strip() for sub in z]
        temp2.extend(z)
    vocabulary.extend(temp2)
    vocabulary = [sub.replace('=', ' ') for sub in vocabulary]
    vocabulary = [sub.replace('-', ' ') for sub in vocabulary]
    vocabulary = [sub.replace('#', '') for sub in vocabulary]
    
    
    recipes = dict()
    count = 1
    for i in aliased_names:
        temp = []
        for ingredient in i:
            ingredient = ingredient.strip()
            for j in vocabulary:
                if(ingredient in j):
                    temp.append(ingredient)
                    break
        temp = list(dict.fromkeys(temp))
        recipes[count] = temp
        count += 1
    return vocabulary,recipes

In [16]:
vocabulary,recipes = fetchIngredients()

In [17]:
with open('culinaryDB_vocabulary.pkl', 'wb') as f:
    pickle.dump(vocabulary, f)
with open('culinaryDB_recipes.pkl', 'wb') as f:
    pickle.dump(recipes, f)

#### reducing culinaryDB

In [88]:
def reduce_vocabulary(vocabulary):
    vocabulary_count = dict()
    for i in vocabulary:
        count = 0
        for j in recipes:
            if i in recipes[j]:
                count += 1
        vocabulary_count[i] = count
    reverse_count = sorted(vocabulary_count.items(), key=lambda item: item[1], reverse = True)
    reverse_vocabulary_count = dict()
    for i in reverse_count:
        if(i[1]>2):
            reverse_vocabulary_count[i[0]] = i[1]
    return list(reverse_vocabulary_count.keys())

In [91]:
new_vocabulary = reduce_vocabulary(vocabulary)
with open('culinaryDB_new_vocabulary.pkl', 'wb') as f:
    pickle.dump(new_vocabulary,f)

In [107]:
with open('culinaryDB_new_recipes.pkl','rb') as f:
    recipe_ids = pickle.load(f)

In [126]:
def get_names_cuisines(recipe_ids)
    data = pd.read_csv('./CulinaryDB/01_Recipe_Details.csv')
    rows = []
    for i in recipe_ids.keys():
        rows.append(data.loc[i-1])
    recipe_names= dict()
    recipe_cuisines = dict()
    for row in rows:
        vals = list(row.values)
        recipe_names[vals[0]] = vals[1]
        recipe_cuisines[vals[0]] = vals[3]
    with open('culinaryDB_new_names.pkl','wb') as f:
        pickle.dump(recipe_names,f)
    with open('culinaryDB_new_cuisines.pkl','wb') as f:
        pickle.dump(recipe_cuisines,f)